In [1]:
#include <iostream>

/*a workaround to solve cling issue*/
#include "../inc/macos_cling_workaround.hpp"
/*set libtorch path, load libs*/
#include "../inc/load_libtorch.hpp"
/*import custom defined macros*/
#include "../inc/custom_def.hpp"
/*import matplotlibcpp*/
#include "../inc/load_matplotlibcpp.hpp"
/*import opencv*/
#include "../inc/load_opencv.hpp"

/*import libtorch header file*/
#include <torch/torch.h>
#include <opencv2/opencv.hpp>
#include <cmath>

// Use (void) to silent unused warnings.
#define assertm(exp, msg) assert(((void)msg, exp))

In [2]:
#define VAR_NAME(Variable) (#Variable)

void print_tensor_size(std::string name, torch::Tensor t)
{
    int dims = t.dim();
    std::cout << name << " dims is (";
    for (int i = 0; i < dims; i++) {
        std::cout << t.size(i);
        if (i < (dims - 1)) std::cout << " x ";
    }
    std::cout << ")" << std::endl;
}

# 卷积神经网络（LeNet）

**LeNet（LeNet-5）由两个部分组成： 卷积编码器和全连接层密集块**

In [3]:
// class Reshape(torch.nn.Module):
//     def forward(self, x):
//         return x.view(-1, 1, 28, 28)

// net = torch.nn.Sequential(Reshape(), nn.Conv2d(1, 6, kernel_size=5,
//                                                padding=2), nn.Sigmoid(),
//                           nn.AvgPool2d(kernel_size=2, stride=2),
//                           nn.Conv2d(6, 16, kernel_size=5), nn.Sigmoid(),
//                           nn.AvgPool2d(kernel_size=2, stride=2), nn.Flatten(),
//                           nn.Linear(16 * 5 * 5, 120), nn.Sigmoid(),
//                           nn.Linear(120, 84), nn.Sigmoid(), nn.Linear(84, 10))

In [4]:
struct Reshape : torch::nn::Module {
  torch::Tensor forward(torch::Tensor X) {
    return X.view({-1, 1, 28, 28});
  }
};

In [5]:
torch::nn::Sequential net(Reshape(),
                          torch::nn::Conv2d(torch::nn::Conv2dOptions(1, 6, 5).padding(2).bias(false)),
                          torch::nn::Sigmoid(),
                          torch::nn::AvgPool2d(torch::nn::AvgPool2dOptions(2).stride(2)),
                          torch::nn::Conv2d(torch::nn::Conv2dOptions(6, 16, 5).bias(false)),
                          torch::nn::Sigmoid(),
                          torch::nn::AvgPool2d(torch::nn::AvgPool2dOptions(2).stride(2)),
                          torch::nn::Flatten(),
                          torch::nn::Linear(16 * 5 * 5, 120),
                          torch::nn::Sigmoid(),
                          torch::nn::Linear(120, 84),
                          torch::nn::Sigmoid(),
                          torch::nn::Linear(84, 10)
                         );

**检查模型**

In [6]:
// X = torch.rand(size=(1, 1, 28, 28), dtype=torch.float32)
// for layer in net:
//     X = layer(X)
//     print(layer.__class__.__name__, 'output shape: \t', X.shape)

In [7]:
auto X = torch::rand({1, 1, 28, 28}, torch::kFloat);

auto& model_ref = *net; 

for (auto module : model_ref)
{
    std::cout << module.ptr()->name() << std::endl;
    X = module.forward(X);
    std::cout << "output shape: \t" << X.sizes() << std::endl;
//     std::cout << module.ptr()->parameters() << std::endl;
}

__cling_N53::Reshape
output shape: 	[1, 1, 28, 28]
torch::nn::Conv2dImpl
output shape: 	[1, 6, 28, 28]
torch::nn::SigmoidImpl
output shape: 	[1, 6, 28, 28]
torch::nn::AvgPool2dImpl
output shape: 	[1, 6, 14, 14]
torch::nn::Conv2dImpl
output shape: 	[1, 16, 10, 10]
torch::nn::SigmoidImpl
output shape: 	[1, 16, 10, 10]
torch::nn::AvgPool2dImpl
output shape: 	[1, 16, 5, 5]
torch::nn::FlattenImpl
output shape: 	[1, 400]
torch::nn::LinearImpl
output shape: 	[1, 120]
torch::nn::SigmoidImpl
output shape: 	[1, 120]
torch::nn::LinearImpl
output shape: 	[1, 84]
torch::nn::SigmoidImpl
output shape: 	[1, 84]
torch::nn::LinearImpl
output shape: 	[1, 10]


**LeNet在Fashion-MNIST数据集上的表现**

In [8]:
// batch_size = 256
// train_iter, test_iter = d2l.load_data_fashion_mnist(batch_size=batch_size)

In [9]:
constexpr uint32_t batch_size = 256;

// Create a multi-threaded data loader for the MNIST dataset.
auto data_loader = torch::data::make_data_loader(
  torch::data::datasets::MNIST("../dataset/fashion_mnist").map(
      torch::data::transforms::Stack<>()),
  /*batch size=*/batch_size);

**对 evaluate_accuracy函数进行轻微的修改**

In [10]:
// def evaluate_accuracy_gpu(net, data_iter, device=None):  
//     """使用GPU计算模型在数据集上的精度。"""
//     if isinstance(net, torch.nn.Module):
//         net.eval()
//         if not device:
//             device = next(iter(net.parameters())).device
//     metric = d2l.Accumulator(2)
//     for X, y in data_iter:
//         if isinstance(X, list):
//             X = [x.to(device) for x in X]
//         else:
//             X = X.to(device)
//         y = y.to(device)
//         metric.add(d2l.accuracy(net(X), y), y.numel())
//     return metric[0] / metric[1]

In [13]:
float evaluate_accuracy_gpu(torch::nn::Sequential net, std::string dataset_path, torch::Device device)
{
    
    
    
    return 0.0;
}

input_line_32:1:34: error: use of undeclared identifier 'data_iter'
float evaluate_accuracy_gpu(net, data_iter, device=None)
                                 ^
input_line_32:1:45: error: use of undeclared identifier 'device'
float evaluate_accuracy_gpu(net, data_iter, device=None)
                                            ^
input_line_32:1:52: error: use of undeclared identifier 'None'
float evaluate_accuracy_gpu(net, data_iter, device=None)
                                                   ^
input_line_32:1:57: error: expected ';' after top level declarator
float evaluate_accuracy_gpu(net, data_iter, device=None)
                                                        ^
                                                        ;


Interpreter Error: 

**为了使用 GPU，我们还需要一点小改动**

In [11]:
// def train_ch6(net, train_iter, test_iter, num_epochs, lr, device):
//     """Train a model with a GPU (defined in Chapter 6)."""
//     def init_weights(m):
//         if type(m) == nn.Linear or type(m) == nn.Conv2d:
//             nn.init.xavier_uniform_(m.weight)

//     net.apply(init_weights)
//     print('training on', device)
//     net.to(device)
//     optimizer = torch.optim.SGD(net.parameters(), lr=lr)
//     loss = nn.CrossEntropyLoss()
//     animator = d2l.Animator(xlabel='epoch', xlim=[1, num_epochs],
//                             legend=['train loss', 'train acc', 'test acc'])
//     timer, num_batches = d2l.Timer(), len(train_iter)
//     for epoch in range(num_epochs):
//         metric = d2l.Accumulator(3)
//         net.train()
//         for i, (X, y) in enumerate(train_iter):
//             timer.start()
//             optimizer.zero_grad()
//             X, y = X.to(device), y.to(device)
//             y_hat = net(X)
//             l = loss(y_hat, y)
//             l.backward()
//             optimizer.step()
//             with torch.no_grad():
//                 metric.add(l * X.shape[0], d2l.accuracy(y_hat, y), X.shape[0])
//             timer.stop()
//             train_l = metric[0] / metric[2]
//             train_acc = metric[1] / metric[2]
//             if (i + 1) % (num_batches // 5) == 0 or i == num_batches - 1:
//                 animator.add(epoch + (i + 1) / num_batches,
//                              (train_l, train_acc, None))
//         test_acc = evaluate_accuracy_gpu(net, test_iter)
//         animator.add(epoch + 1, (None, None, test_acc))
//     print(f'loss {train_l:.3f}, train acc {train_acc:.3f}, '

**训练和评估LeNet-5模型**

In [12]:
// lr, num_epochs = 0.9, 10
// train_ch6(net, train_iter, test_iter, num_epochs, lr, d2l.try_gpu())